In [1]:
!pip install deap

In [13]:
import random
import numpy as np
from deap import base, creator, tools, algorithms

# -----------------------------------
# STEP 1: Problem Definition
# -----------------------------------
# We need to optimize a delivery route by minimizing total travel distance.
# The warehouse is at index 0, and we must visit all locations exactly once.

# Distance matrix representing the travel cost between locations
distance_matrix = np.array([
    [0, 10, 15, 20, 25, 30],  # Warehouse
    [10, 0, 35, 25, 30, 20],  # Location 1
    [15, 35, 0, 30, 20, 25],  # Location 2
    [20, 25, 30, 0, 15, 10],  # Location 3
    [25, 30, 20, 15, 0, 35],  # Location 4
    [30, 20, 25, 10, 35, 0]   # Location 5
])

num_locations = len(distance_matrix)  # Total locations (including warehouse)
population_size = 10   # Number of candidate solutions in each generation
generations = 100      # Number of iterations
mutation_rate = 0.2    # Probability of mutation

# -----------------------------------
# STEP 2: Genetic Algorithm Setup
# -----------------------------------
# We define a DEAP "FitnessMin" class to represent our objective (minimize distance).
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Minimize distance
creator.create("Individual", list, fitness=creator.FitnessMin)  # A delivery route is a list of locations

toolbox = base.Toolbox()

# Function to generate a valid random route (excluding warehouse)
def create_valid_route():
    """Creates a shuffled list of locations excluding the warehouse (index 0)."""
    route = list(range(1, num_locations))  # Locations 1 to num_locations-1
    random.shuffle(route)  # Shuffle locations randomly
    return creator.Individual(route)  # Return as DEAP Individual

toolbox.register("individual", tools.initIterate, creator.Individual, create_valid_route)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# -----------------------------------
# STEP 3: Fitness Function (Objective Function)
# -----------------------------------
# The fitness function calculates the total travel distance for a given route.
def route_distance(individual):
    """Computes total travel distance for a given route."""
    route = [0] + individual + [0]  # Start and end at the warehouse
    total_distance = sum(distance_matrix[route[i], route[i + 1]] for i in range(len(route) - 1))
    return total_distance,  # Return as a tuple (DEAP requires tuple format)

toolbox.register("evaluate", route_distance)


# -----------------------------------
# TODO: Extend the Lab (Student Task)
# -----------------------------------
# Modify the fitness function to include time constraints.
# Steps:
# 1. Assume each location has a preferred delivery time slot (e.g., [2, 4, 1, 3, 5]).
# 2. Penalize routes that visit locations later than their required time.
# 3. Modify the `route_distance` function to include a penalty.
# 4. Re-run the genetic algorithm and analyze changes in the best route.


# -----------------------------------
# STEP 4: Genetic Algorithm Operators
# -----------------------------------
# Crossover: Uses Partially Mapped Crossover (PMX) to swap sections between two parents
def custom_pmx(ind1, ind2):
    """Applies PMX crossover after converting to 0-based indices, then converts back."""
    ind1[:] = [x - 1 for x in ind1]  # Convert to 0-based indices
    ind2[:] = [x - 1 for x in ind2]

    tools.cxPartialyMatched(ind1, ind2)  # Apply PMX crossover

    ind1[:] = [x + 1 for x in ind1]  # Convert back to 1-based indices
    ind2[:] = [x + 1 for x in ind2]
    return ind1, ind2  # Return modified individuals

toolbox.register("mate", custom_pmx)  # Crossover function
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=mutation_rate)  # Mutation: Swap elements
toolbox.register("select", tools.selTournament, tournsize=3)  # Selection: Tournament method

# -----------------------------------
# STEP 5: Genetic Algorithm Execution
# -----------------------------------
def genetic_algorithm():
    """Executes the Genetic Algorithm to find the optimal delivery route."""
    population = toolbox.population(n=population_size)  # Create initial population

    # Run the genetic algorithm for a set number of generations
    for generation in range(generations):
        offspring = algorithms.varAnd(population, toolbox, cxpb=0.7, mutpb=mutation_rate)  # Apply crossover & mutation
        population = toolbox.select(offspring, k=len(population))  # Select best individuals for next generation

    # Return the best solution found
    best_route = tools.selBest(population, k=1)[0]  # Get best route
    best_distance = route_distance(best_route)[0]  # Compute total distance
    return best_route, best_distance

# Run the genetic algorithm
best_route, best_distance = genetic_algorithm()

# -----------------------------------
# STEP 6: Print Results
# -----------------------------------
print("Best Delivery Route:", [0] + best_route + [0])  # Add warehouse start/end
print("Minimum Distance:", best_distance)


Best Delivery Route: [0, 2, 5, 3, 4, 1, 0]
Minimum Distance: 105


In [17]:
import random
import numpy as np
from deap import base, creator, tools, algorithms

# -----------------------------------
# STEP 1: Problem Definition
# -----------------------------------
# Distance matrix representing the travel cost between locations
distance_matrix = np.array([
    [0, 10, 15, 20, 25, 30],  # Warehouse
    [10, 0, 35, 25, 30, 20],  # Location 1
    [15, 35, 0, 30, 20, 25],  # Location 2
    [20, 25, 30, 0, 15, 10],  # Location 3
    [25, 30, 20, 15, 0, 35],  # Location 4
    [30, 20, 25, 10, 35, 0]   # Location 5
])

num_locations = len(distance_matrix)  # Total locations (including warehouse)
population_size = 10   # Number of candidate solutions in each generation
generations = 100      # Number of iterations
mutation_rate = 0.2    # Probability of mutation

# Preferred delivery times (in hours) for locations 1 to 5 (index 1 to 5)
preferred_times = [2, 4, 1, 3, 5]  # List of preferred delivery times for locations

# -----------------------------------
# STEP 2: Genetic Algorithm Setup
# -----------------------------------
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Minimize distance
creator.create("Individual", list, fitness=creator.FitnessMin)  # A delivery route is a list of locations

toolbox = base.Toolbox()

# Function to generate a valid random route (excluding warehouse)
def create_valid_route():
    """Creates a shuffled list of locations excluding the warehouse (index 0)."""
    route = list(range(1, num_locations))  # Locations 1 to num_locations-1
    random.shuffle(route)  # Shuffle locations randomly
    return creator.Individual(route)  # Return as DEAP Individual

toolbox.register("individual", tools.initIterate, creator.Individual, create_valid_route)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# -----------------------------------
# STEP 3: Modified Fitness Function (with time constraints)
# -----------------------------------
def route_distance(individual):
    """Computes total travel distance with time constraint penalties."""
    route = [0] + individual + [0]  # Start and end at the warehouse
    total_distance = 0
    time_penalty = 0
    current_time = 0  # Start at time 0

    for i in range(len(route) - 1):
        travel_time = distance_matrix[route[i], route[i + 1]]  # Travel time = distance
        current_time += 1  # Assume 1 time unit per stop (modify if needed)
        total_distance += travel_time

        # Apply penalty if arrival time exceeds preferred time slot
        if current_time > preferred_times[route[i + 1] - 1]:  # Use route[i + 1] - 1 as the index
            time_penalty += (current_time - preferred_times[route[i + 1] - 1]) * 10  # Example penalty factor

    # Return tuple with total distance and penalty
    return total_distance + time_penalty,  # DEAP requires tuple format

toolbox.register("evaluate", route_distance)

# -----------------------------------
# STEP 4: Genetic Algorithm Operators
# -----------------------------------
def custom_pmx(ind1, ind2):
    """Applies PMX crossover after converting to 0-based indices, then converts back."""
    ind1[:] = [x - 1 for x in ind1]  # Convert to 0-based indices
    ind2[:] = [x - 1 for x in ind2]

    tools.cxPartialyMatched(ind1, ind2)  # Apply PMX crossover

    ind1[:] = [x + 1 for x in ind1]  # Convert back to 1-based indices
    ind2[:] = [x + 1 for x in ind2]
    return ind1, ind2  # Return modified individuals

toolbox.register("mate", custom_pmx)  # Crossover function
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=mutation_rate)  # Mutation: Swap elements
toolbox.register("select", tools.selTournament, tournsize=3)  # Selection: Tournament method

# -----------------------------------
# STEP 5: Genetic Algorithm Execution
# -----------------------------------
def genetic_algorithm():
    """Executes the Genetic Algorithm to find the optimal delivery route."""
    population = toolbox.population(n=population_size)  # Create initial population

    # Run the genetic algorithm for a set number of generations
    for generation in range(generations):
        offspring = algorithms.varAnd(population, toolbox, cxpb=0.7, mutpb=mutation_rate)  # Apply crossover & mutation
        population = toolbox.select(offspring, k=len(population))  # Select best individuals for next generation

    # Return the best solution found
    best_route = tools.selBest(population, k=1)[0]  # Get best route
    best_distance = route_distance(best_route)[0]  # Compute total distance
    return best_route, best_distance

# Run the genetic algorithm
best_route, best_distance = genetic_algorithm()

# -----------------------------------
# STEP 6: Print Results
# -----------------------------------
print("Best Delivery Route:", [0] + best_route + [0])  # Add warehouse start/end
print("Minimum Distance (with penalties):", best_distance)


Best Delivery Route: [0, 5, 2, 4, 3, 1, 0]
Minimum Distance (with penalties): 195


In [21]:
import random
import numpy as np
from deap import base, creator, tools, algorithms

# -----------------------------------
# STEP 1: Problem Definition
# -----------------------------------
# Distance matrix representing the travel cost between locations
distance_matrix = np.array([
    [0, 10, 15, 20, 25, 30],  # Warehouse
    [10, 0, 35, 25, 30, 20],  # Location 1
    [15, 35, 0, 30, 20, 25],  # Location 2
    [20, 25, 30, 0, 15, 10],  # Location 3
    [25, 30, 20, 15, 0, 35],  # Location 4
    [30, 20, 25, 10, 35, 0]   # Location 5
])

num_locations = len(distance_matrix)  # Total locations (including warehouse)
population_size = 10   # Number of candidate solutions in each generation
generations = 100      # Number of iterations
mutation_rate = 0.2    # Probability of mutation

# Preferred delivery times (in hours) for locations 1 to 5 (index 1 to 5)
preferred_times = [2, 4, 1, 3, 5]  # List of preferred delivery times for locations

# -----------------------------------
# STEP 2: Genetic Algorithm Setup
# -----------------------------------
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Minimize distance
creator.create("Individual", list, fitness=creator.FitnessMin)  # A delivery route is a list of locations

toolbox = base.Toolbox()

# Function to generate a valid random route (excluding warehouse)
def create_valid_route():
    """Creates a shuffled list of locations excluding the warehouse (index 0)."""
    route = list(range(1, num_locations))  # Locations 1 to num_locations-1
    random.shuffle(route)  # Shuffle locations randomly
    return creator.Individual(route)  # Return as DEAP Individual

toolbox.register("individual", tools.initIterate, creator.Individual, create_valid_route)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# -----------------------------------
# STEP 3: Modified Fitness Function (with time constraints)
# -----------------------------------
def route_distance(individual):
    """Computes total travel distance with time constraint penalties."""
    route = [0] + individual + [0]  # Start and end at the warehouse
    total_distance = 0
    time_penalty = 0
    current_time = 0  # Start at time 0

    for i in range(len(route) - 1):
        travel_time = distance_matrix[route[i], route[i + 1]]  # Travel time = distance
        current_time += 1  # Assume 1 time unit per stop (modify if needed)
        total_distance += travel_time

        # Apply penalty if arrival time exceeds preferred time slot
        if current_time > preferred_times[route[i + 1] - 1]:  # Use route[i + 1] - 1 as the index
            time_penalty += (current_time - preferred_times[route[i + 1] - 1]) * 10  # Example penalty factor

    # Return tuple with total distance and penalty
    return total_distance + time_penalty,  # DEAP requires tuple format

toolbox.register("evaluate", route_distance)

# -----------------------------------
# STEP 4: Genetic Algorithm Operators
# -----------------------------------
def custom_pmx(ind1, ind2):
    """Applies PMX crossover after converting to 0-based indices, then converts back."""
    ind1[:] = [x - 1 for x in ind1]  # Convert to 0-based indices
    ind2[:] = [x - 1 for x in ind2]

    tools.cxPartialyMatched(ind1, ind2)  # Apply PMX crossover

    ind1[:] = [x + 1 for x in ind1]  # Convert back to 1-based indices
    ind2[:] = [x + 1 for x in ind2]
    return ind1, ind2  # Return modified individuals

toolbox.register("mate", custom_pmx)  # Crossover function
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=mutation_rate)  # Mutation: Swap elements
toolbox.register("select", tools.selTournament, tournsize=3)  # Selection: Tournament method

# -----------------------------------
# STEP 5: Genetic Algorithm Execution
# -----------------------------------
def genetic_algorithm():
    """Executes the Genetic Algorithm to find the optimal delivery route."""
    population = toolbox.population(n=population_size)  # Create initial population

    # Run the genetic algorithm for a set number of generations
    for generation in range(generations):
        offspring = algorithms.varAnd(population, toolbox, cxpb=0.7, mutpb=mutation_rate)  # Apply crossover & mutation
        population = toolbox.select(offspring, k=len(population))  # Select best individuals for next generation

    # Return the best solution found
    best_route = tools.selBest(population, k=1)[0]  # Get best route
    best_distance = route_distance(best_route)[0]  # Compute total distance
    return best_route, best_distance

# Run the genetic algorithm
best_route, best_distance = genetic_algorithm()

# -----------------------------------
# STEP 6: Print Results
# -----------------------------------
print("Best Delivery Route:", [0] + best_route + [0])  # Add warehouse start/end
print("Minimum Distance (with penalties):", best_distance)


Best Delivery Route: [0, 2, 1, 3, 5, 4, 0]
Minimum Distance (with penalties): 195


In [ ]:
Analysis Question                 
1.Before modifying the fitness function, what was the optimal route and distance?
This would be the optimal route and distance with no penalty applied for time violations.

The output of the program will give the optimal route and the corresponding distance (before penalties are included).
Route Change:
After introducing time constraints, the route may change because the genetic algorithm now considers time penalties in addition to minimizing travel distance. Locations visited after their preferred delivery time will incur a penalty, which can affect the order in which locations are visited to minimize these penalties.

Cost Increase:
Yes, the cost (i.e., the total fitness score) is likely to increase due to the addition of time penalties. Routes that are visited later than their preferred times will incur a penalty, which raises the total cost. However, the genetic algorithm might still find an efficient route that balances both distance and time constraints.
                                                                                      
2.After including time constraints, how did the route change? Did the cost increase?
Route Change: The algorithm may modify the order of locations to ensure that the locations are visited before their preferred times, which could result in a different route.

Cost Increase: The introduction of time penalties likely increases the cost of the optimal route. Routes that violate time constraints will incur additional penalties, increasing the total cost of the solution. You might find that the best route is still efficient in terms of distance, but the total cost will be higher due to time constraints.
                                                                                      
3. What happens when you increase the number of generations? What happens when you update the mutation rate, or change the size of the population? Briefly explain the observations. 
Increasing the Number of Generations: Increasing the number of generations gives the algorithm more opportunities to refine the solution. The route might become more optimized as the population evolves over time, and the cost may decrease with more generations. However, if the algorithm already converges to a good solution early, increasing generations too much could lead to diminishing returns.

Updating the Mutation Rate: Increasing the mutation rate allows for more diversity in the population, preventing the algorithm from getting stuck in local minima. This could result in more varied solutions and potentially better routes. However, too much mutation might destabilize the algorithm, while too little mutation could cause it to converge too early and miss better solutions.

Changing the Population Size: Increasing the population size typically provides more candidate solutions, which can help the algorithm find better solutions by maintaining diversity. However, a larger population requires more computational resources and may slow down the algorithm. A smaller population may result in faster computation but risks finding suboptimal solutions.